In [1]:
%load_ext autoreload
#from google.colab import drive
#drive.mount('/content/drive')

#! pip install transformers datasets pytorch-lightning
# Use widgets to add a form that turns off/on dev
# Use nbdev to generate notebooks
# import ipywidgets as widgets 
import os
#os.chdir("/content/drive/MyDrive/ldh")
os.chdir("/Users/danielpham/Google Drive/ldh")

## Data Preparation
- Data is loaded through LDHDataModule. 
- Returned 

In [2]:
%autoreload
from src.LDHDataModule import LDHDataModule
from src.LightningReapp import LightningReapp

ldhdata = LDHDataModule(batch_size=16)

Loading cached processed dataset at /Users/danielpham/Google Drive/ldh/training/obj/cache-40db00b5805d18ed.arrow
Training data loaded from disk.


In [ ]:
import pytorch_lightning as lit
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

trainer = lit.Trainer(
    track_grad_norm=2, # Tracks L2 NORM  
    max_epochs=50,
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    # gpus=1,
    fast_dev_run=2,
    weights_summary="top",
    callbacks=[EarlyStopping(monitor='val_loss')]
  )

# Fit model
# trainer.tune(system, ldhdata.train_dataloader())
# #trainer.setup_trainer(system) #Sanity check 
# trainer.fit(system, ldhdata)
for i in range(5):
  ldhdata.current_split = i
  system = LightningReapp()
  print(f"=== Running Split {i} ===")
  trainer.fit(system, ldhdata)

# Test Dataset